# Parameters List
1. k_batch=list(np.arange(1.0, 4.0, 0.5)) # k parameter

2. k_instance=list(np.arange(1.0, 4.0, 0.5)) # This can be set the same as k_batch and hence excluded

3. count_threshold_measure=['median','max'] # alpha parameter

4. logical_parameter=['and','or']



In [1]:
import numpy as np
k_batch=list(np.arange(1.0, 4.5, 0.5))
count_threshold_measure=['median','max']
logical_parameter=['and','or']

In [2]:
import itertools

hyperparameter_space = {
    'k_batch': k_batch,
    'count_threshold_measure':count_threshold_measure,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))


In [3]:
all_combinations

[(1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max', 'and'),
 (3.0, 'max', 'or'),
 (3.5, 'median', 'and'),
 (3.5, 'median', 'or'),
 (3.5, 'max', 'and'),
 (3.5, 'max', 'or'),
 (4.0, 'median', 'and'),
 (4.0, 'median', 'or'),
 (4.0, 'max', 'and'),
 (4.0, 'max', 'or')]

In [4]:
len(all_combinations)

28

### Design the Process flow for grid search

1. Autoencoder is pre-trained , which will be used for all combinations ,this will reduce the processing overhead
2. The validation data needs to be processed once 
3. The iterations will start from threshold computations 
4. The objective is least warnings and detections on normal data set , least detection delay , and more warnnings after detection point
5. We can set the normal data size 20 batches , drifted data size also 20 batches , both combined with drift starting from batch 20
6. We can assign weights to factors in (5) and define a score



In [5]:
total_batches=40
batch_size=32
drift_point=20

WnDs=[i for i in range(0,21)]
print("Warnings in Drift Stream before drift point",WnDs)
DnDs=[i for i in range(0,21)]
print("Detections in Drift Stream before drift point",DnDs)
delay=[ i for i in range(21)]
print("Detection Delay:",delay)
DdDs=[i for i in range(19)]
print("Detections after drift point:",DdDs)
WdDs=[i for i in range(19)]
print("Warnings after drift point:",WdDs)

Warnings in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detection Delay: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Warnings after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [6]:
WnDS=[i for i in range(0,21)]
DnDs=[i for i in range(0,21)]
delay=[ i for i in range(21)]
DdDs=[i for i in range(19)]
WdDs=[i for i in range(19)]

In [7]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    best_value = float('-inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value > best_value:
                            best_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, best_value

# Find the best parameters and the corresponding value
best_parameters, best_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Best parameters:', best_parameters)
print('Value:', best_value)

Best parameters: (0, 0, 0, 18, 18)
Value: 54


In [8]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    worst_value = float('inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value < worst_value:
                            worst_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, worst_value

# Find the best parameters and the corresponding value
best_parameters, worst_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Worst parameters:', best_parameters)
print('Value:', worst_value)

Worst parameters: (20, 20, 20, 0, 0)
Value: -120


# Dataset 

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing


from skmultiflow.data import HyperplaneGenerator
from scipy.stats import norm
import random
from scipy import stats 


import tensorflow
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import anderson

In [10]:
data=pd.read_csv('vd_nondrifted.csv')

In [11]:
data

,col1,col2,col3,col4,col5,class
0,7.000000,6.000000,8.000000,6.000000,5.000000,1
1,-2.729465,0.395907,-1.244328,1.058099,0.717900,0
2,-0.144599,-0.698764,5.386546,0.453879,0.461973,0
3,8.000000,8.000000,4.000000,4.000000,6.000000,1
4,3.000000,4.000000,5.000000,9.000000,5.000000,1
...,...,...,...,...,...,...
19195,6.000000,2.000000,3.000000,7.000000,7.000000,1
19196,7.000000,9.000000,6.000000,6.000000,7.000000,1
19197,4.000000,5.000000,7.000000,5.000000,6.000000,1
19198,0.666549,-0.170226,3.552791,-2.485941,3.502892,0


In [12]:
drifted_data=pd.read_csv('vd_drifted.csv')

In [13]:
drifted_data

,col1,col2,col3,col4,col5,class
0,-7.120779,-0.743991,-1.436859,3.409486,0.263934,0
1,0.950664,-4.606839,-0.759950,3.226979,-1.076545,0
2,4.000000,3.000000,2.000000,5.000000,6.000000,1
3,0.445997,0.395188,-0.676667,-2.993964,0.629553,0
4,-2.335979,1.226460,-0.295903,0.687385,0.872043,0
...,...,...,...,...,...,...
955,2.166315,-0.442264,0.154055,-1.231357,-2.908712,0
956,-2.159207,7.446561,1.351154,-3.486939,0.257229,0
957,-0.375560,-0.460687,2.260007,-2.052887,-0.642849,0
958,1.889754,-2.195848,-1.364404,0.715772,0.872968,0


In [14]:
del drifted_data['class']

In [15]:
drifted_data=drifted_data[0:1280]

## 3. Data Preprocessing 
1. Initial 70% data is used for training ( training and validation for AE)
2. Next 20 % data is used as a validation set to compute thresholds
3. Next 10 % data is used as a test stream initilally without drift 
  

In [16]:
def normalize_encode_split(data,label_col,pos_val,neg_val):        
    # Apply Label Encoding 
    # Apply MinMAx Normalization
    # Split into training  , test (validation set 1) and stream ( non-drifted data stream)   
            
    train=data[0:int(len(data)*0.70)]
    test=data[int((0.70*len(data))):int((0.90*len(data)))]
    stream=data[int((0.90*len(data))):len(data)]
    
    train_positives = train[train[label_col] == pos_val]
    train_negatives = train[train[label_col] == neg_val]
    
    
    y_positive=train_positives[label_col] 
    y_negative=train_negatives[label_col]
    X_positive=train_positives.drop([label_col],axis=1)
    X_negative=train_negatives.drop([label_col],axis=1)
    
    
    return train, test, X_positive,X_negative , stream ,y_positive ,y_negative  

In [17]:
train, test, X_positive,X_negative , stream, y_positive ,y_negative =normalize_encode_split(data,'class',1,0)

### Best Autoencoder Hyperparameters will be deternmined for each data set separetely 

In [18]:
def autoencoder (train,epochs,val_set):
    
    # set bottleneck 1/3 of the input layer size
    #encoding_dim = int(np.floor(train.shape[-1]/3))
       
    # input layer size= # of attributes in the dataset after one-hot encoding
    input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
    encoded = layers.Dense(6, activation='relu')(input_layer) # Code Layer 1
    encoded=layers.Dense(2,activation='relu')(encoded) # Bottleneck
    decoded=layers.Dense(6,activation='relu')(encoded)# Decode Layer 1
    decoded = layers.Dense(train.shape[1], activation='sigmoid')(decoded) # Output Layer
          
    autoencoder = tensorflow.keras.Model(input_layer, decoded)
    # above model maps an input to its reconstruction 
   
    autoencoder.compile(optimizer='adam', loss='mse') # Train autoencoder 
    history=autoencoder.fit(train,train, 
                epochs=epochs,
                batch_size=32,
                shuffle=True,
                validation_data=(val_set, val_set)).history
    return  autoencoder , history

In [19]:
def train_encoders(X_Positive,X_Negative, epochs):
    
    X_Positive_train=X_Positive[0:int(len(X_Positive)*0.90)]
    X_Positive_test=X_Positive[int((0.90*len(X_Positive))):len(X_Positive)-1]
    
    X_Negative_train=X_Negative[0:int(len(X_Negative)*0.90)]
    X_Negative_test=X_Negative[int((0.90*len(X_Negative))):len(X_Negative)-1]

    print("Training Autoencoder on Positive Examples ")
    encoder_pos_class, history_positive_class=autoencoder(X_Positive_train,epochs,X_Positive_test)
    print("Training Autoencoder on Negative Examples ")
    encoder_neg_class,history_negative_class=autoencoder(X_Negative_train,epochs,X_Negative_test)
    
    return encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class

In [20]:
encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class=train_encoders(X_positive,X_negative, 100)

Training Autoencoder on Positive Examples 
Epoch 1/100
190/190 [==============================] - 19s 97ms/step - loss: 21.5833 - val_loss: 19.0937
Epoch 2/100
190/190 [==============================] - 1s 3ms/step - loss: 18.8028 - val_loss: 18.9143
Epoch 3/100
190/190 [==============================] - 2s 11ms/step - loss: 18.5536 - val_loss: 18.9121
Epoch 4/100
190/190 [==============================] - 1s 3ms/step - loss: 18.7491 - val_loss: 18.9118
Epoch 5/100
190/190 [==============================] - 2s 11ms/step - loss: 18.7111 - val_loss: 18.9116
Epoch 6/100
190/190 [==============================] - 1s 3ms/step - loss: 18.6232 - val_loss: 18.9115
Epoch 7/100
190/190 [==============================] - 2s 11ms/step - loss: 18.6375 - val_loss: 18.9115
Epoch 8/100
190/190 [==============================] - 0s 3ms/step - loss: 18.6992 - val_loss: 18.9115
Epoch 9/100
190/190 [==============================] - 2s 11ms/step - loss: 18.7238 - val_loss: 18.9115
Epoch 10/100
190/190 [==

190/190 [==============================] - 1s 3ms/step - loss: 18.7141 - val_loss: 18.9114
Epoch 80/100
190/190 [==============================] - 2s 11ms/step - loss: 18.7236 - val_loss: 18.9114
Epoch 81/100
190/190 [==============================] - 1s 3ms/step - loss: 18.8171 - val_loss: 18.9114
Epoch 82/100
190/190 [==============================] - 2s 11ms/step - loss: 18.5851 - val_loss: 18.9114
Epoch 83/100
190/190 [==============================] - 1s 3ms/step - loss: 18.5908 - val_loss: 18.9114
Epoch 84/100
190/190 [==============================] - 2s 11ms/step - loss: 18.6125 - val_loss: 18.9114
Epoch 85/100
190/190 [==============================] - 1s 3ms/step - loss: 18.6453 - val_loss: 18.9114
Epoch 86/100
190/190 [==============================] - 2s 11ms/step - loss: 18.7062 - val_loss: 18.9114
Epoch 87/100
190/190 [==============================] - 1s 3ms/step - loss: 18.7355 - val_loss: 18.9114
Epoch 88/100
190/190 [==============================] - 2s 11ms/step - lo

189/189 [==============================] - 1s 3ms/step - loss: 2.6585 - val_loss: 2.6957
Epoch 59/100
189/189 [==============================] - 2s 11ms/step - loss: 2.7512 - val_loss: 2.6912
Epoch 60/100
189/189 [==============================] - 0s 2ms/step - loss: 2.6493 - val_loss: 2.6888
Epoch 61/100
189/189 [==============================] - 2s 11ms/step - loss: 2.6655 - val_loss: 2.6873
Epoch 62/100
189/189 [==============================] - 1s 3ms/step - loss: 2.6163 - val_loss: 2.6871
Epoch 63/100
189/189 [==============================] - 2s 11ms/step - loss: 2.7019 - val_loss: 2.6858
Epoch 64/100
189/189 [==============================] - 1s 3ms/step - loss: 2.6866 - val_loss: 2.6877
Epoch 65/100
189/189 [==============================] - 2s 11ms/step - loss: 2.6734 - val_loss: 2.6863
Epoch 66/100
189/189 [==============================] - 1s 3ms/step - loss: 2.6083 - val_loss: 2.6857
Epoch 67/100
189/189 [==============================] - 2s 11ms/step - loss: 2.6659 - val_l

In [21]:
test

,col1,col2,col3,col4,col5,class
13440,5.000000,6.000000,7.000000,5.000000,4.000000,1
13441,5.000000,7.000000,5.000000,5.000000,7.000000,1
13442,-0.650654,-1.808092,-0.204549,-0.091213,0.301353,0
13443,5.000000,4.000000,6.000000,5.000000,5.000000,1
13444,2.000000,5.000000,6.000000,5.000000,7.000000,1
...,...,...,...,...,...,...
17275,-2.104131,1.072304,-4.488367,0.192177,-0.265180,0
17276,-2.333141,-1.837311,0.573894,-0.991890,1.587727,0
17277,6.000000,4.000000,6.000000,5.000000,4.000000,1
17278,0.939109,0.269401,0.474452,2.012080,0.158114,0


## Validation Data

In [22]:
def make_batches(test_data): 
        data=np.array(test_data)
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches

In [23]:
def process_validation_data(test,encoder_pos_class,encoder_neg_class,batch_size=32):
   
    # From the test data ,separate positive and negative class data and remove the class column

    test_pos_class=test[test['class']==1]
    test_neg_class=test[test['class']==0]
    del test_pos_class['class']
    del test_neg_class['class']
       
       
    # This function make batches of the data stream
     
    def make_batches(test_data):
        data=np.array(test_data)
        batch_size=32
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches
    #
    
    batches_pos=make_batches(test_pos_class)
    batches_neg=make_batches(test_neg_class)
    
    # This function computes reconconstruction error for each instance as well as average reco.error of a batch passed to it. 
    def compute_instance_loss_batch_loss(batch,batch_size,encoder):
        mse_list=[]
        mse_sum=0  
        for i in range(0,batch.shape[0]):
            ROW = np.array([batch[i]])
            pred= encoder.predict(ROW) 
            mse = np.mean(np.power(batch[i] - pred, 2))
            mse_list.append(mse)
            mse_sum+=mse
        avg_mse=mse_sum/batch_size
        return mse_list,avg_mse   

            
    # This function computes recon.errr of all the batches .  
    def process_all_batches(batches,encoder):
        batch_avg_mse=[]
        batch_mse_values={}
        for b in batches:
            print(b,end=" ")
            mse_list,average_mse=compute_instance_loss_batch_loss(batches[b],batch_size,encoder)
            batch_avg_mse.append(average_mse)
            batch_mse_values[b]=mse_list
        return batch_avg_mse,batch_mse_values

    batch_avg_mse_pos,batch_mse_values_pos=process_all_batches(batches_pos,encoder_pos_class)
    batch_avg_mse_neg,batch_mse_values_neg=process_all_batches(batches_neg,encoder_neg_class)
       
    
    
    return  batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg
    
    
    
   
    

In [24]:
batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg=process_validation_data(test,encoder_pos_class,encoder_neg_class)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 

In [25]:
k_parameter=list(np.arange(1.0, 4.5, 0.5))
alpha_parameter=['median','max']
logical_parameter=['and','or']
import itertools

hyperparameter_space = {
    'k': k_parameter,
    'count_threshold_measure':alpha_parameter,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))
all_combinations

[(1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max', 'and'),
 (3.0, 'max', 'or'),
 (3.5, 'median', 'and'),
 (3.5, 'median', 'or'),
 (3.5, 'max', 'and'),
 (3.5, 'max', 'or'),
 (4.0, 'median', 'and'),
 (4.0, 'median', 'or'),
 (4.0, 'max', 'and'),
 (4.0, 'max', 'or')]

In [26]:
len(all_combinations)

28

### AEDDM Framwework

In [27]:
def detect_at_batch_level(test_batch,b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list):
    
    # Layer 1 Variables 
    mse_list_layer1=[]     # Holds the recon loss values predicted by Layer 1 Autoencoder for a single batch
    exceed_count_layer1=0  # How many instances exceed layer one instance level threshold
    mse_sum=0              # sum of recon.error values from layer 1 AE for this batch
    
    
    
    # Layer 2 Variables 
    mse_list_layer2=[]      # Holds the recon.error values predicted by Layer 2 Autoencoder for a single batch
    exceed_count_layer2=-1  # If a batch is not passed to the layer2 AE , then exceed count value will be -1 . This is symbolic.
    mse_sum_layer2=0        # sum of recon.error values from layer 2 AE for this batch
    
        
    layer1_excede_list=[] # Holds the batch numbers of bathces exceeding layer1 thresholds
    #all_excede_list=[]    # Hold the batch number of batches exceeding both Layer-1 and Layer-2 AE Thresholds 
    layer_one_instance_exceed_list=[] # Holds the indices of instances exceeding layer 1 instance threshold
    layer_two_instance_exceed_list=[] # Holds the indices of instances exceeding layer 2 instance thresholds
     
      
    # Determine Layer 1 and Layer AE and their associated thresholds
    layer_one_batch_thres= batch_thres_pos if batch_thres_pos<batch_thres_neg  else batch_thres_neg
    layer_two_batch_thres= batch_thres_pos if batch_thres_pos>batch_thres_neg  else batch_thres_neg
    layer_one_encoder= encoder_pos_class if batch_thres_pos<batch_thres_neg else encoder_neg_class
    layer_two_encoder= encoder_pos_class if batch_thres_pos>batch_thres_neg else encoder_neg_class
    
    layer_one_count_threshold=count_thresh_pos if batch_thres_pos<batch_thres_neg  else count_thres_neg
    layer_two_count_threshold=count_thresh_pos if batch_thres_pos>batch_thres_neg  else count_thres_neg
    
    layer1_ins_thresh=instance_thresh_pos if  batch_thres_pos<batch_thres_neg  else instance_thresh_neg
    layer2_ins_thresh=instance_thresh_pos if  batch_thres_pos>batch_thres_neg  else instance_thresh_neg
    
    # Pass each instance of a batch to Layer 1 AE. Compute Batch MSE and Number of Instances excceding Layer 1 Instance Thresh       
    avg_mse_layer1=0
    for i in range(0,test_batch.shape[0]):
        ROW = np.array([test_batch[i]])
        pred= layer_one_encoder.predict(ROW) 
        mse = np.mean(np.power(test_batch[i] - pred, 2))
        mse_list_layer1.append(mse)
        if mse>layer1_ins_thresh:
            exceed_count_layer1+=1
            layer_one_instance_exceed_list.append(i)
        mse_sum+=mse    
    avg_mse_layer1=(mse_sum)/len(test_batch)
    avg_mse_layer2=0 
    # Check if This batch exceeds both Layer 1 batch and count thresholds
    if ((avg_mse_layer1>layer_one_batch_thres) and ( exceed_count_layer1 >layer_one_count_threshold) ):
        layer1_excede_list.append(b)# Keep track of batches exceesding layer 1 thresholds
        exceed_count_layer2=0
        # Pass each instance of this batch to Layer 2 AE. Compute Batch MSE and Number of Instances excceding Layer 2 Instance Thresh
        for i in range(0,test_batch.shape[0]):
            ROW = np.array([test_batch[i]])
            pred= layer_two_encoder.predict(ROW) 
            mse = np.mean(np.power(test_batch[i] - pred, 2))
            mse_list_layer2.append(mse)
            if mse>layer2_ins_thresh:
                exceed_count_layer2+=1
                layer_two_instance_exceed_list.append(i)
            mse_sum_layer2+=mse    
        avg_mse_layer2=(mse_sum_layer2)/len(test_batch)
        
        if  (avg_mse_layer2 > layer_two_batch_thres) and (exceed_count_layer2>layer_two_count_threshold):
            all_excede_list.append(b)# Keep track of batches exceesding layer 2 thresholds
          
    return  all_excede_list,mse_list_layer1 , exceed_count_layer1 ,avg_mse_layer2,exceed_count_layer2,len(layer_two_instance_exceed_list),layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list,layer_two_instance_exceed_list,avg_mse_layer1 

In [28]:
def detect_stream_drift(batches,encoder_pos_class,encoder_neg_class,batch_thres_pos,batch_thres_neg, count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg,logical):
    exceed_count_layer2_instance_thresh={} # Holds Number of instances exceeding thresholds of Both Layer 1 and layer 2 AE.
    
    mse_dict_L1={}         # Holds batchwise recon.error values from Layer 1 AE
    exceed_count_L1={}     # Batchwise number of Instances exceeding layer 1 count threshold
    layer_one_instance_exceed_list={} # batch wise list of instances ( indices ) exceeding Layer 1 count threshold
    avg_mse_l1={}
    
    mse_dict_L2={}        # Holds batchwise recon. error  values from Layer 2 AE
    exceed_count_L2={}    # Batchwise number of Instances exceeding layer 2 count threshold
    layer_two_instance_exceed_list={} # For each batch maintains the indices where recon.error exceeds layer 2 threshold
    avg_mse_l2={}         #
    all_excede_list=[]
    n=0
    for b in batches:
        print("\n\n")
        print("**************")
        print('\nBatch Number : {}'.format(b))
        all_excede_list,mse_dict_L1[b],exceed_count_L1[b],avg_mse_l2[b],exceed_count_L2[b],exceed_count_layer2_instance_thresh[b],layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list[b],layer_two_instance_exceed_list[b],avg_mse_l1[b]= detect_at_batch_level(batches[b],b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list)
             
        
        print('\nData Points Exceeding Layer 1 Encoder Instance Threshold : {} '.format(layer_one_instance_exceed_list[b]))
        print('\nData Points Exceeding Layer 2 Encoder Instance Threshold: {}'.format(layer_two_instance_exceed_list[b])) 
        print('\nNumber of Data Points Exceeding Layer 2 Encoder Instance thresholds: {}'.format(exceed_count_layer2_instance_thresh[b]))
        
    mse_list_layer1=return_list_of_dict_values(avg_mse_l1)
    exceed_count_list_layer1=return_list_of_dict_values(exceed_count_L1)
    
    
    ## Detect Drift at Batch Level 
    print ("\n Drift Detection at Batch Level\n")
    exceed_list=return_list_of_dict_values(exceed_count_L2)
    mse_list=return_list_of_dict_values(avg_mse_l2)
    score,col_values=detect_drift(mse_list,exceed_list,layer_two_batch_thres,layer_two_count_threshold,logical)
    
    
    return score,col_values,all_excede_list,exceed_count_layer2_instance_thresh ,exceed_count_L2,avg_mse_l2 ,mse_list_layer1,exceed_count_list_layer1




In [29]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    score=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={score}")
    
    col_values=[WnDs,DnDs,delay,WdDs,DdDs,score]
    
    return score,col_values


In [30]:
results_df_p=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [31]:
# This function computes how many instances in a batch exceed instance threshold
def threshold_exceed_count(batch_mse_values, thr):
    exceed_count={}
    for key in batch_mse_values.keys():
        count=0
        list=batch_mse_values[key]
        for a in range(0,len(list)):
            if list[a]>thr:
                count+=1
        exceed_count[key]=count 
    values = exceed_count.values()
    
    return exceed_count

def compute_count_threshold(exceed_count_dict,alpha):
    exceed_list=[]
    for key in exceed_count_dict.keys():
        exceed_list.append(exceed_count_dict[key]) 
    #print(" from with in count threshold")
    #print(alpha)
    method = getattr(np,alpha)  # this enables np.median() or np.max() based on alpha value
    count_threshold=method(exceed_list)    
    return count_threshold
   
def compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse,batch_avg_mse_values,k,alpha,N=60):
        # Computes Instance Threshold
        zscore_list=[]
        for i in range(0,N):
            value_list=batch_avg_mse_values[i]

            mean=np.mean(value_list)
            sigma=np.std(value_list)
            thres_zscore=mean+k*sigma
            zscore_list.append(thres_zscore)
        # Computes batch threshold
        mean_batch=np.mean(batch_avg_mse)
        sigma_batch=np.std(batch_avg_mse)    
        batch_thresh=mean_batch+k*sigma_batch
        
        method = getattr(np,alpha)
        instance_thresh = method(zscore_list)
        
        return  instance_thresh, zscore_list , batch_thresh


    
# Make batches of the normal data stream     
s=stream.copy()
del s['class']
batches_n=make_batches(s)
#print(batches_n)
# For testing purpose choose 20 batches only
batches_n=dict(itertools.islice(batches_n.items(), 20))
    
# Drifted Data Stream
drifted=drifted_data[0:1280]
batches_d=make_batches(drifted)    
    
    
    
def return_list_of_dict_values(d):
    values_list=[]
    for key in d.keys():
        values_list.append(d[key])
    return values_list
    

        
def pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg):
    # Define the parametric space
    k_parameter=list(np.arange(0.5, 4.5, 0.5))
    alpha_parameter=['median','max']
    logical_parameter=['and','or']
    results={}
   
    global results_df_p
    row=0
    
    
    for k in k_parameter:
        for alpha in alpha_parameter:
            for logical in logical_parameter:
                print("********************************")
                
                print(k,alpha,logical)
                values=[k, alpha, logical]
                parameters=(k,alpha,logical)
                instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
                instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
                #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
                # Counts the MSE values exceeding threshold in each batch 
                exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
                exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
                count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
                count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
                print("Thresholds for Positive AE")
                #print(alpha)
                print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
                print("Thresholds for Nagative AE")
                print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
                # AEDDM Starts Here 
                
                thres_zscore_batch_pos=batch_thresh_pos
                thres_zscore_batch_neg=batch_thresh_neg
                             
                
                #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
                    #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
                print(" Drifted Data Processing Starts Here")
                
                score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    
        
                results[parameters]=score
                values.extend(col_values)
                results_df_p = results_df_p.append(pd.Series(values, index=results_df_p.columns), ignore_index=True)
                print(results_df_p)
    
    return results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d

In [32]:
results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d=pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg)

********************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 19.1855806866513 , Count Threshold: 9.0
Thresholds for Nagative AE
Batch Thresold: 2.883483740388379 , Count Threshold: 7.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 7, 8, 12, 13, 16, 19, 21, 23, 27, 28, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [21]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 9, 10, 11, 12, 13, 14, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 20, 23, 25, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 7, 9, 10, 11, 12,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 21, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 21, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 5, 8, 9, 10, 11, 12, 13, 16, 18, 21, 22, 23, 24, 26, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 9, 16, 24]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 18

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 5, 7, 8, 9, 10, 12, 15, 17, 19, 20, 22, 23, 24, 25, 26, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 8, 10, 12, 15, 17, 19, 24, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 9



**************

Batch Number : 19

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 7, 8, 9, 10, 13, 15, 16, 17, 20, 23, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [10, 13, 17, 20, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 20

Data Points Ex


Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 6, 7, 8, 10, 11, 12, 14, 15, 18, 19, 22, 23, 26, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [8]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 8

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 4, 5, 7, 8, 9, 12, 16, 17, 18, 21, 26, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 4, 7, 9, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 7, 8, 11, 13, 14, 15, 16, 17, 18, 19, 21, 22, 25, 27, 28, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [7, 16, 18, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 3, 4, 6, 8, 9, 10, 11, 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 7, 8, 12, 13, 16, 21, 27, 28, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [21]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 6, 9, 11, 12, 13, 14, 20, 21, 23, 24, 25, 26, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 23, 25]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 7, 9, 11, 12, 13, 16, 17, 18, 20, 23, 24, 25, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [12, 13, 23]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 4, 6, 8, 10, 14, 15, 16, 18, 20, 24, 26, 

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 5, 6, 7, 16, 21, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 6, 9, 11, 12, 14, 20, 23, 24, 25, 26, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 7, 9, 11, 12, 13, 16, 17, 18, 20, 23, 24, 25] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 8, 10, 14, 15, 16, 18, 20, 24, 26, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold:


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14

 Drift Detection at Batch Level

 Threshold exceeds at batch : 26
[26]
Warning Level at Batch 26
 Threshold exceeds at batch : 27
[26, 27]
Warning Level at Batch 27
 Threshold exceeds at batch : 28
[26, 27, 28]
 Drift Confirmed at Batch No :  26
 Threshold exceeds at batch : 29
[26, 27, 28, 29]
 Drift Confirmed at Batch No :  27
Number of Drifted Batches2
All Drifted Batches:  [26, 27]
Drifted Batches before Drift Point:  []
Drifted Batches after Drift Point:  [26, 27]
WnDs=0 ,WdDs=2 , DnDs=0,DdDs=2 , delay= 6 , value=108
      k   alpha logical WnDs DnDs delay WdDs DdDs score
0   0.5  median     and    0    0     3    5    5   126
1   0.5  median      or    0    0     0    8    8   144
2  


Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 10, 13, 15, 16, 17, 20, 23, 26, 27, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 20

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 21

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20] 

Data Points Excee


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 4, 7, 9, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 14, 15, 16, 18, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 8, 17, 20, 23, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 14, 17, 19, 21, 23, 25, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



*******


Data Points Exceeding Layer 1 Encoder Instance Threshold : [21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 20, 23, 24, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 12, 13, 23, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [8, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceed

In [33]:
results_df_p

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.5,median,and,0,0,3,5,5,126
1,0.5,median,or,0,0,0,8,8,144
2,0.5,max,and,0,0,3,5,5,126
3,0.5,max,or,0,0,1,7,7,138
4,1.0,median,and,0,0,4,4,4,120
5,1.0,median,or,0,0,0,8,8,144
6,1.0,max,and,0,0,4,4,4,120
7,1.0,max,or,0,0,2,6,6,132
8,1.5,median,and,0,0,5,3,3,114
9,1.5,median,or,0,0,1,7,7,138


In [34]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [35]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Categorical, Real


def aeddm_function(x):
    global results_df
    row=0
    k,alpha,logical=x
    values=[k, alpha, logical]
    #results_df.iloc[row :3] = values
        
    print("***************************************************")
    print(k,alpha,logical)
    parameters=(k,alpha,logical)
    instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
    instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
    #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
    # Counts the MSE values exceeding threshold in each batch 
    exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
    exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
    count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
    count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
    print("Thresholds for Positive AE")
    #print(alpha)
    print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
    print("Thresholds for Nagative AE")
    print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
    # AEDDM Starts Here 
                
    thres_zscore_batch_pos=batch_thresh_pos
    thres_zscore_batch_neg=batch_thresh_neg
                             
                
    #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
        #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
             #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    print(" Drifted Data Processing Starts Here")
                
    score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    #value+=100
    print(col_values)
    print(values)
    values.extend(col_values)
    print(values)
    
    results_df = results_df.append(pd.Series(values, index=results_df.columns), ignore_index=True)
    print(results_df)
    
    values=[]
    col_values=[]
    print(k,alpha,logical)
    return score 



In [36]:
# Define the search space
row=0
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 1 to 4
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    #aeddm_function,
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=20,  # Number of iterations
    acq_func='EI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 24.414674705041122 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 4.447409501727323 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 5, 6, 7, 16, 21, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 6, 9, 11, 12, 14, 20, 23, 24, 25, 26, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 7, 9, 11, 12, 13, 16, 17, 18, 20, 23, 24, 25] 

Data Po


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14

 Drift Detection at Batch Level

 Threshold exceeds at batch : 26
[26]
Warning Level at Batch 26
 Threshold exceeds at batch : 27
[26, 27]
Warning Level at Batch 27
 Threshold exceeds at batch : 28
[26, 27, 28]
 Drift Confirmed at Batch No :  26
 Threshold exceeds at batch : 29
[26, 27, 28, 29]
 Drift Confirmed at Batch No :  27
Number of Drifted Batches2
All Dri


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [9, 12]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 10, 12, 13]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 9, 10, 11]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 3, 7, 9, 10, 13, 15, 16, 17, 20, 23, 26, 27, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [10, 13, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 20

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 19, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 10, 14]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 21

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 8]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 8, 11, 13, 17, 20, 23, 25, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 14, 17, 19, 21, 23, 25, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 12

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 6, 10, 18, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 13

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 9, 10, 21, 24, 26, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instanc

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 21, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 21, 24, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 23, 24, 25, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 9, 10, 11, 12, 15, 16, 17, 19, 21, 22, 23, 25, 26, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 21, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 15

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 4, 5, 6, 7, 8, 10, 15, 17, 18, 20, 21, 23, 26, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 7, 20, 23, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 16

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 4, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 21, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 6, 16, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 6, 9, 11, 12, 14, 20, 21, 23, 24, 25, 26, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 7, 9, 11, 12, 13, 16, 17, 18, 20, 23, 24, 25] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 8, 10, 14, 15, 16, 18, 20, 24, 26, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 4, 5, 8, 9, 10, 12, 14, 17, 18, 20, 23, 24, 25, 27, 28, 30] 

Data Points Exce


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 24, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16

 Drift Detection at Batch Level

 Threshold exceeds at batch : 13
[13]
Warning Level at Batch 13
 Threshold exceeds at batch : 19
[19]
Warning Level at Batch 19
 Threshold exceeds at batch : 21
[21]
Warning Level at Batch 21
 Threshold exceeds at batch : 22
[21, 22]
Warning Level at Batch 22
 Threshold exceeds at batch : 23
[21, 22, 23]
 Drift Confirmed at Batch No :  21
 Threshold exceeds at batch : 24
[21, 22, 23, 24]
 Drift Confirmed at Batch No :  22
 Threshold exceeds at batch : 25
[21, 22, 23, 24, 25]
 Drift Confirmed at Batch No :  23
 Threshold exceeds at batch : 26
[21, 22, 23, 24, 25, 26]
 Drift Confirmed at Batch No :  24
 Threshold exceeds at batch : 27
[21, 22, 2


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 4, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 21, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 6, 16, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 5, 8, 9, 10, 11, 12, 13, 16, 18, 21, 22, 23, 24, 26, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 9, 16, 24]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 18

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 5, 7, 8, 9, 10, 12, 15, 17, 19, 20, 22, 23, 24, 25, 26, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 8, 10, 12, 15, 17, 19, 24, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 9



**************

Batch Number : 19

Data Points Exceeding L

In [37]:
df_EI=results_df.copy()

In [38]:
df_EI

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,6,2,2,108
1,3.996642,max,and,0,0,6,2,2,108
2,1.857688,median,or,1,1,0,8,8,141
3,3.462088,max,or,0,0,5,3,3,114
4,2.052085,max,or,0,0,2,6,6,132
5,3.698867,max,and,0,0,6,2,2,108
6,3.786947,median,or,0,0,2,6,6,132
7,3.309651,max,or,0,0,3,5,5,126
8,3.527571,median,or,0,0,1,7,7,138
9,1.455675,max,or,0,0,2,6,6,132


In [39]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [40]:
# Define the search space
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 1 to 4
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=20,  # Number of iterations
    acq_func='PI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 24.414674705041122 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 4.447409501727323 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 5, 6, 7, 16, 21, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 6, 9, 11, 12, 14, 20, 23, 24, 25, 26, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 7, 9, 11, 12, 13, 16, 17, 18, 20, 23, 24, 25] 

Data Po


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14

 Drift Detection at Batch Level

 Threshold exceeds at batch : 26
[26]
Warning Level at Batch 26
 Threshold exceeds at batch : 27
[26, 27]
Warning Level at Batch 27
 Threshold exceeds at batch : 28
[26, 27, 28]
 Drift Confirmed at Batch No :  26
 Threshold exceeds at batch : 29
[26, 27, 28, 29]
 Drift Confirmed at Batch No :  27
Number of Drifted Batches2
All Dri


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [9, 12]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 10, 12, 13]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 9, 10, 11]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 3, 7, 9, 10, 13, 15, 16, 17, 20, 23, 26, 27, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [10, 13, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 20

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 19, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 10, 14]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 21

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 8]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 8, 11, 13, 17, 20, 23, 25, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 14, 17, 19, 21, 23, 25, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 12

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 6, 10, 18, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 13

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 9, 10, 21, 24, 26, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instanc

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 21, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 21, 24, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 23, 24, 25, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 9, 10, 11, 12, 15, 16, 17, 19, 21, 22, 23, 25, 26, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 21, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 15

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 4, 5, 6, 7, 8, 10, 15, 17, 18, 20, 21, 23, 26, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 7, 20, 23, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 16

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 4, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 21, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 6, 16, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 9, 10, 11, 12, 13, 14, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 20, 23, 25, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 7, 9, 10, 11, 12, 13, 16, 17, 18, 20, 21, 23, 24, 25, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 12, 13, 23, 24]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 4, 6, 7, 8, 10, 14, 15, 16, 18, 20, 24, 26, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [8, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 21, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 21, 24, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 23, 24, 25, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Number of Data Points


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 9, 10, 11, 12, 15, 16, 17, 19, 21, 22, 23, 25, 26, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 21, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 15

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 4, 5, 6, 7, 8, 10, 15, 17, 18, 20, 21, 23, 26, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 7, 20, 23, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 16

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 4, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 21, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 6, 16, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4,

In [41]:
max_value = result.fun  # Access the maximum function value
print("Maximum value:", max_value)
print("Best parameters:", result.x)

Maximum value: -144
Best parameters: [1.2027544713023057, 'median', 'or']


In [42]:
results_df

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,6,2,2,108
1,3.996642,max,and,0,0,6,2,2,108
2,1.857688,median,or,1,1,0,8,8,141
3,3.462088,max,or,0,0,5,3,3,114
4,2.052085,max,or,0,0,2,6,6,132
5,3.698867,max,and,0,0,6,2,2,108
6,3.786947,median,or,0,0,2,6,6,132
7,3.309651,max,or,0,0,3,5,5,126
8,3.527571,median,or,0,0,1,7,7,138
9,1.455675,max,or,0,0,2,6,6,132


In [43]:
Thresh=0.212964
count_thresh=0
logical='and'

In [44]:
drift_point=20
WnDs= 0 
WdDs=0 
DnDs=0          
delay=None     
DdDs=0   

In [45]:
def compute_delay(drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift2(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(drift_batches,drift_batches_after_drift_point)
    value=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={value}")
    
    

In [46]:
detect_drift2(avg_mse_l2_list_d,exceed_count_L2_countThresh_d, Thresh,count_thresh,logical)

 Threshold exceeds at batch : 25
[25]
Warning Level at Batch 25
 Threshold exceeds at batch : 26
[25, 26]
Warning Level at Batch 26
 Threshold exceeds at batch : 27
[25, 26, 27]
 Drift Confirmed at Batch No :  25
 Threshold exceeds at batch : 28
[25, 26, 27, 28]
 Drift Confirmed at Batch No :  26
 Threshold exceeds at batch : 29
[25, 26, 27, 28, 29]
 Drift Confirmed at Batch No :  27
Number of Drifted Batches3
All Drifted Batches:  [25, 26, 27]
Drifted Batches before Drift Point:  []
Drifted Batches after Drift Point:  [25, 26, 27]
WnDs=0 ,WdDs=3 , DnDs=0,DdDs=3 , delay= 5 , value=114
